# Create a structured database for the Iris dataset

### Common Imports

In [1]:
import sys
from pathlib import Path

# TODO: move common imports to vars
# sys.path.append(str(Path(__file__).parent))
# from vars import *
from sys import platform
MONGO_DB_NAME = "iris-db"
DB_IP = "10.171.18.104"

if platform == "linux":
    DB_IP = "localhost"

from pymongo import MongoClient, collection, ASCENDING

In [2]:
def ensure_exists(path):
    if not path.exists():
        path.mkdir(parents=True, exist_ok=True)

In [3]:
DB_BASE_ = Path("~/datasets/iris-db/").expanduser()
ensure_exists(DB_BASE_)

In [4]:
META_COLL = "meta"

## WORKING FOR CASIA-V1

### Copying to structured folder and adding to mongodb as well

In [5]:
# DB_BASE_
DB_NAME = "CASIA-V1"
DB_NAME_ = Path(DB_NAME)
DB_PATH_ = DB_BASE_ / DB_NAME_
ensure_exists(DB_PATH_)

META_DB = "meta-db"

In [6]:
orig_db_base_ = Path("~/datasets/iris_datasets/CASIA/V1/CASIA-IrisV1/CASIA-IrisV1/CASIA Iris Image Database (version 1.0)").expanduser()

In [7]:
# connect with authentication and authdb as admin

# con=MongoClient(DB_IP, username="admin", password="Temppass@123", authSource="admin")


In [8]:

# coll=con[MONGO_DB_NAME][META_COLL]
# # insert into db
# desc = """
# CASIA Iris Image Database Version 1.0 (CASIA-IrisV1) includes 756 iris images
# from 108 eyes. For each eye, 7 images are captured in two sessions with our
# self-developed device CASIA close-up iris camera (Fig.1), where three samples are
# collected in the first session (Fig.2(a)) and four in the second session (Fig.2(b)). All
# images are stored as BMP format with resolution 320*280
# In order to protect our IPR in the design of our iris camera (especially the NIR
# illumination scheme), the pupil regions of all iris images in CASIA-IrisV1 were
# automatically detected and replaced with a circular region of constant intensity to
# mask out the specular reflections from the NIR illuminators. Such editing
# clearly makes iris boundary detection much easier but has minimal or no effects on
# other components of an iris recognition system, such as feature extraction and
# classifier design.
# It is suggested that you compare two samples from the same eye taken in different
# sessions when you want to compute the within-class variability. For example, the iris
# images in the first session can be employed as training dataset and those from the
# second session are used for testing.
# """
# metadata = {
#     "db_id": DB_NAME,
#     "name": "CASIA-IrisV1",
#     "desc": desc,
#     "specs":{
#         "num_images": 756,
#         "num_people": 108,
#         "num_eyes": 108,
#         "num_samples_per_eye": 7,
#         "num_sessions": 2,
#         "image_format": "bmp",
#         "image_resolution": "320x280",
#         "width": 320,
#         "height": 280
#     }
# }
# coll.insert_one(metadata)


In [9]:
# create an unqiue index with db_id
# coll.create_index([("db_id", ASCENDING)], unique=True)

In [10]:
# class IRIS_DB:
#     def __init__(self, mongo_db_name=MONGO_DB_NAME, mongo_colleciton=META_COLL):
#         self.conn = MongoClient(DB_IP, username="admin", password="Temppass@123", authSource="admin")[mongo_db_name][db_name]

#     def insert_metadata(self, metadata):
#         self.conn.insert_one(metadata)

#     def find_metadata(self, db_id):
#         return self.conn.find_one({"db_id": db_id})


In [ ]:
# Setup logging
from lib import LoggerManager as lm, FileManager as fm, logging
from functools import lru_cache
from time import sleep
# logger = LoggerManager.get_logger(name=Path(__file__).stem)
lg = lm.get_logger(__name__)
# set log level to info
lg.setLevel(logging.INFO)

class IRIS_DB:
    """
    In: None
    Out: IRIS_DB Object that can be used to connect to a particular db
    param: 
        dbcoll - name of the db collection
    """

    def __init__(
        self, 
        db_ip=None, 
        mongo_db_name=None, 
        meta_coll=None
        # db_coll=None, # this could be used to shortcircuit the process?
        ) -> object:
        self.db_ip = DB_IP if db_ip is None else db_ip
        self.mongo_db_name = MONGO_DB_NAME if mongo_db_name is None else mongo_db_name
        self.meta_coll = META_COLL if meta_coll is None else meta_coll
        # self.db_coll = DB_COLL if db_coll is None else db_coll
        ## get user:passwd from mongo_creds.txt file
        user,passwd = fm.read_creds()
        self._mongo_admin_user = user
        self._mongo_admin_password = passwd
        self.closing = False
        
    @property
    @lru_cache(maxsize=None) # Caches the result after the first call
    def mongo_client(self):
        """Lazily creates and returns the MongoClient instance."""
        if self.closing:
            delattr(self, 'mongo_client')
            return None
        lg.debug("MongoDB client is not initialized. Creating client...")
        mc = MongoClient(
            self.db_ip,
            username=self._mongo_admin_user,
            password=self._mongo_admin_password,
            authSource="admin"
        )
        lg.debug("MongoDB client created successfully.")
        return mc
    
    @property
    @lru_cache(maxsize=None) # Caches the result after the first call
    def mongo_conn(self):
        """Lazily creates and returns the Database object using the client."""
        if self.closing:
            delattr(self, 'mongo_conn')
            return None
        lg.debug("Establishing MongoDB database connection...")
        # This will automatically trigger the mongo_client property if needed
        conn = self.mongo_client[self.mongo_db_name]
        lg.debug("MongoDB connection established successfully.")
        return conn
    
    @property
    # @lru_cache(maxsize=None) # Caches the result after the first call
    def avail_ds(self) -> set:
        """Lazily creates and returns the set of available iris-db using the mongo client."""
        lg.debug("Establishing MongoDB database connection...")
        # This will automatically trigger the mongo_client property if needed
        avail_ds = {i['db_id'] for i in self.mongo_conn[self.meta_coll].find({}, {'_id': 0, 'db_id': 1})}
        lg.debug("Fetched List of avail databases.")
        return avail_ds

    

    def get_avail_ds(self) -> set:
        """Get a set of available IRIS db_id."""
        return self.avail_ds
    # alias
    get_datasets = get_avail_ds

    def __enter__(self):
        """Called when entering the 'with' statement."""
        lg.debug("Entering context...")
        return self # Return the instance to be used in the 'with' block
    
    def __exit__(self,*args):
        """Called when exiting the 'with' statement."""
        # This method is always called, ensuring the connection is closed.
        self.close()
        lg.debug("MongoClient connection closed.")

    def find_db(self, db_name, avail_ds=None, acc=0.4, count=1) -> str:
        from difflib import get_close_matches
        # 1. Create a mapping from lowercase name to original name.
        mapping = {db.lower(): db for db in (avail_ds or self.avail_ds)}

        # 2. Get the lowercase versions of all available DBs for matching.
        lower_avail_ds = list(mapping.keys())
        
        # 3. Perform the match on the lowercase versions.
        matches = get_close_matches(db_name.lower(), lower_avail_ds, n=count, cutoff=acc)

        # 4. If a match is found, use the mapping to return the original name.
        if matches:
            if count > 1:
                return [mapping[match] for match in matches]
            return mapping[matches[0]]

    def connect(self, ds_name, fuzzy=0.4) -> collection:
        """Will connect to the database into a given collection"""
        try:
            avail_ds = self.avail_ds
        except Exception as e:
            lg.error(f"Error accessing available databases: {e}")
            return None
        

        if (closest_match := self.find(ds_name, avail_ds)):
            self.ds_name = closest_match
            lg.info(f"Connecting to {self.ds_name} Collection")
            # self.ds_prefix=Path(DS_PREFIX)
            # self.ds_path=self.ds_prefix/self.ds_name
        else:
            lg.info(f"{ds_name} Collection Not found in available datasets. List: {avail_ds}")
        return self.mongo_conn[self.ds_name]
    
    
    # def get_data_from_mongodb(dbName,query,proj):
#     ds=Datasets()
#     db=ds.connect(dbName)
#     docs=list(db.find(query,proj))
#     del ds
#     return docs

# def update_data_to_mongodb(dbName,docs):
#     ds=Datasets()
#     db=ds.connect(dbName)
#     for doc in docs:
#         db.update_one({'_id':doc['_id']},{'$set':doc})
#     # db.update
#     del ds
    
    
    
    def close(self):
        """Explicitly close the mongo client connection"""        
        try:
            self.closing = True
            if hasattr(self, 'mongo_client'):
                lg.debug("checking for mongoclient")
                self.mongo_client.close()
                lg.info("MongoDB client connection closed successfully.")
            else:
                lg.debug("MongoDB client was not initialized; no connection to close.")
        except Exception as e:
            lg.error(f"Error closing MongoDB client connection: {e}")
    
    def __del__(self):
        try:
            if self.closing:
                return
            self.close()
            lg.debug("MongoDB connection closed successfully.")
        except Exception as e:
            lg.error(str(e))
            
# def get_data_from_mongodb(dbName,query,proj):
#     ds=Datasets()
#     db=ds.connect(dbName)
#     docs=list(db.find(query,proj))
#     del ds
#     return docs

# def update_data_to_mongodb(dbName,docs):
#     ds=Datasets()
#     db=ds.connect(dbName)
#     for doc in docs:
#         db.update_one({'_id':doc['_id']},{'$set':doc})
#     # db.update
#     del ds
# db=IRIS_DB()

# # print(db.connect('casiav3'))
# # sleep(1)
# db.close()

2025-08-28 15:07:48,281 - lib - connect - INFO - Connecting to CASIA-V1 Collection
2025-08-28 15:07:48,282 - lib - close - INFO - MongoDB client connection closed successfully.


Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin'), 'iris-db'), 'CASIA-V1')


In [ ]:
with IRIS_DB() as db:
    # Use the db object to interact with the database
    print(db.list_ds())
    # print(db.mongo_conn[db.meta_coll].find({}, {'_id': 0, 'db_id': 1}))
    
    pass

2025-08-28 14:35:23,648 - lib - read_creds - DEBUG - MongoDB credentials loaded successfully.
2025-08-28 14:35:23,649 - lib - __enter__ - DEBUG - Entering context...
2025-08-28 14:35:23,649 - lib - avail_db - DEBUG - Establishing MongoDB database connection...
2025-08-28 14:35:23,649 - lib - mongo_conn - DEBUG - Establishing MongoDB database connection...
2025-08-28 14:35:23,650 - lib - mongo_client - DEBUG - MongoDB client is not initialized. Creating client...
2025-08-28 14:35:23,653 - lib - mongo_client - DEBUG - MongoDB client created successfully.
2025-08-28 14:35:23,654 - lib - mongo_conn - DEBUG - MongoDB connection established successfully.
2025-08-28 14:35:23,674 - lib - avail_db - DEBUG - MongoDB connection established successfully.
2025-08-28 14:35:23,675 - lib - close - DEBUG - checking for mongoclient
2025-08-28 14:35:23,676 - lib - close - DEBUG - MongoDB client connection closed successfully.
2025-08-28 14:35:23,676 - lib - __exit__ - DEBUG - MongoClient connection close

{'CASIA-V1'}
